In [2]:
%load_ext jupyter_black

In [1]:
import json
from glob import glob
from pathlib import Path

import nvector
import numpy as np
import pandas as pd
import geopandas as gpd
# ml

import tensorflow
from sklearn.manifold import LocallyLinearEmbedding
# plotting
import matplotlib.pyplot as plt



all_files = sorted(glob("/workspaces/sppp/data/*"))

2022-07-19 14:00:33.723891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-19 14:00:33.723933: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [17]:

def open_file(filepath: Path):
    with filepath.open("rb") as fin:
        return json.load(fin)

def pre():
    ...


moisture_indicies = [
    "EBSHEAR",
    "SRH01KM",
    "MEANWIND_1-3kmAGL",
    "MESH",
    "VIL_DENSITY",
    "FLASH_RATE",
    "FLASH_DENSITY",
    "MAXLLAZ",
    "P98LLAZ",
    "P98MLAZ",
    "MAXRC_EMISS",
    "MAXRC_ICECF",
    "WETBULB_0C_HGT",
    "PWAT",
    "CAPE_M10M30",
    "LJA",
    "SIZE",
    "AVG_BEAM_HGT",
    "MOTION_EAST",
    "MOTION_SOUTH",
    "PS",
    "ID",
    "centroid",
]
stablitity_indicies = [
    "MUCAPE",
    "MLCAPE",
    "MLCIN",
]
lift_indicies = []
growth_indicies = []

lle = LocallyLinearEmbedding(n_components=3, n_neighbors=2)
from typing import NewType
FeatureCollection = NewType("FeatureCollection",dict)
def to_dataframe(fc:FeatureCollection)->pd.DataFrame:
    df = gpd.GeoDataFrame.from_features(fc["features"])
    df["validTime"] = pd.to_datetime(fc["validTime"], format="%Y%m%d_%H%M%S %Z")
    df["centroid"] = df["geometry"].centroid
    
    return df.set_index(["validTime", "ID"])


def to_midf()->pd.DataFrame:

    def generate():
        for file in all_files:
            fc = open_file(Path(file))
            yield to_dataframe(fc)
    return pd.concat(generate())

midf = to_midf()
midf

geometry  \
validTime                 ID                                                         
2021-10-11 00:00:53+00:00 89234  POLYGON ((-96.06000 38.40000, -95.99000 38.400...   
                          89321  POLYGON ((-80.66000 24.76000, -80.62000 24.760...   
                          89467  POLYGON ((-96.98000 36.80000, -96.87000 36.800...   
                          89470  POLYGON ((-75.15000 35.44000, -75.11000 35.440...   
                          89519  POLYGON ((-98.29000 35.59000, -98.27000 35.590...   
...                                                                            ...   
2021-10-11 01:00:58+00:00 90389  POLYGON ((-97.10000 37.72000, -97.05000 37.720...   
                          90390  POLYGON ((-98.03000 34.15000, -97.99000 34.100...   
                          90391  POLYGON ((-98.87000 31.84000, -98.85000 31.830...   
                          90392  POLYGON ((-99.03000 31.68000, -99.01000 31.680...   
                          90393  POLYGON ((-99.35000 31.38000, -99.31000 31.380...   

                                MUCAPE MLCAPE MLCIN EBSHEAR SRH01KM  \
validTime                 ID                                          
2021-10-11 00:00:53+00:00 89234    485      0     0    48.6      63   
                          89321   2691   2179    -2    22.9      14   
                          89467   1367    937   -19    61.9     148   
                          89470   2493   2076     0    24.0      19   
                          89519   2035   1656   -17    74.6     197   
...                                ...    ...   ...     ...     ...   
2021-10-11 01:00:58+00:00 90389    576      0     0    52.1      74   
                          90390   2113   2067   -31    59.3     192   
                          90391   2362   1910   -22    67.5     197   
                          90392   2208   1744   -15    66.8     170   
                          90393   1812   1428   -32    58.1      88   

                                MEANWIND_1-3kmAGL  MESH VIL_DENSITY  \
validTime                 ID                                          
2021-10-11 00:00:53+00:00 89234              12.9  0.25        1.67   
                          89321               4.0  0.43        1.60   
                          89467              17.6  1.43        2.85   
                          89470              16.3  0.05        0.62   
                          89519              26.4  2.19        3.29   
...                                           ...   ...         ...   
2021-10-11 01:00:58+00:00 90389               8.4  0.00        0.52   
                          90390              36.4  0.25        0.88   
                          90391              32.8  0.04        0.77   
                          90392              32.2  0.59        1.75   
                          90393              31.5  0.48        1.68   

                                FLASH_RATE  ... WETBULB_0C_HGT PWAT  \
validTime                 ID                ...                       
2021-10-11 00:00:53+00:00 89234         10  ...           11.8  1.5   
                          89321          7  ...           11.2  1.4   
                          89467         60  ...           11.9  1.7   
                          89470          0  ...           14.3  2.3   
                          89519        181  ...           11.7  1.9   
...                                    ...  ...            ...  ...   
2021-10-11 01:00:58+00:00 90389          0  ...           11.7  1.5   
                          90390          4  ...           12.2  1.9   
                          90391          0  ...           11.7  1.6   
                          90392         12  ...           11.8  1.5   
                          90393         15  ...           11.4  1.2   

                                CAPE_M10M30  LJA  SIZE         AVG_BEAM_HGT  \
validTime                 ID                                                  
2021-10-11 00:00:53+00:00

In [19]:
import gym
class AirSimCarEnv(gym.Env):
    def __init__(self):
        # ..
        # image_shape is (height, width, channels).
        self.observation_space = spaces.Box(low=0, high=255, shape=image_shape, dtype=np.uint8)
        #I wrote a CarAgent class to control car with AirSim APIs
        self.car_agent = CarAgent()
    def step(self, action):
        # action is produced by DQN, action is discrete
        self.car_agent.move(action)
        # compute reward based on state(position) of the car
        car_state= self.car_agent.getCarState()
        reward = self._compute_reward(car_state)
        # check if the episode is done
        car_controls = self.car_agent.getCarControls()
        done = self._isDone(car_state, car_controls, reward)
        # log info
        info = {}
        # observation is RGB image from car's camera
        observation = self.car_agent.observe()
        return observation, reward, done, info

# env = gym.make("TRACK")

def train():
    ...


NameNotFound: Environment TRACK doesn't exist. 